In [1]:
using System;
using System.Collections.Generic;
using System.Net.Http;
using System.Text.Json;
using System.Threading.Tasks;
using System.IO;

In [2]:
string parentPath = "https://raw.githubusercontent.com/abhijeetunreal/3DB/refs/heads/main/model";

In [ ]:
static async Task<List<string>> ListFilesAsync(HttpClient http, string owner,string repo, string path)
{
    var result = new List<string>();
    var apiUrl = $"https://api.github.com/repos/{owner}/{repo}/contents/{path}";
    http.DefaultRequestHeaders.UserAgent.ParseAdd("CSharpApps"); 

    var resp = await http.GetStringAsync(apiUrl);
    using var doc = JsonDocument.Parse(resp);

    foreach (var item in doc.RootElement.EnumerateArray())
    {
        var type = item.GetProperty("type").GetString();
        if (type == "file")
        {
            // raw URL:
            var rawUrl = item.GetProperty("download_url").GetString();
            result.Add(rawUrl);
        }
        else if (type == "dir")
        {
            // recurse into subdirectory
            var subPath = item.GetProperty("path").GetString();
            var subUrls = await ListFilesAsync(http, owner, repo, subPath);
            result.AddRange(subUrls);
        }
    }

    return result;
}

In [ ]:
// Change these to your target repo/owner/path:
const string Owner  = "abhijeetunreal";
const string Repo   = "3DB";
const string Path   = "model"; 
var http = new HttpClient();

var urls = await ListFilesAsync(http, Owner, Repo, Path);
foreach (var u in urls)
    Console.WriteLine(u);

In [ ]:
public class DataRequestModel
{
    public bool IsFavourite { get; set; } = false;
    public string Name { get; set; } = "CFD Models Name";
    public string Thumbnail { get; set; } = "https://codefrydev.in/images/IconCodefrydev.svg";
    public string Description { get; set; } = "A Nerve Plant";

    public string Source { get; set; } =
        "https://raw.githubusercontent.com/abhijeetunreal/3DB/refs/heads/main/model/nerveplant/nerveplant.glb";

    public string Poster { get; set; } = "poster.webp";
    public string AltText { get; set; } = "3D Model";
    public string IosSrc { get; set; } = "";

    public List<string> AnimationNames { get; set; } =[];

    public List<string> EnvironmentImages { get; set; } =["https://modelviewer.dev/shared-assets/environments/spruit_sunrise_1k_HDR.jpg"];

    public List<string> SkyboxImages { get; set; } =["https://modelviewer.dev/shared-assets/environments/spruit_sunrise_1k_HDR.jpg"];
}

In [ ]:
var listOfContent = new Dictionary<string,DataRequestModel>();
foreach (var u in urls)
{
    var fileName = u.Split('/').Last();
    var folderName = u.Split('/')[u.Split('/').Length - 2]; 
    listOfContent.TryAdd(folderName, new DataRequestModel());
    listOfContent[folderName].Name = folderName;

    if(fileName.Contains("poster"))
    {
        listOfContent[folderName].Poster = fileName;
    }
    else if (fileName.Contains("ios"))
    {
        listOfContent[folderName].IosSrc = fileName;
    }
    else if (fileName.Contains("alt"))
    {
        listOfContent[folderName].AltText = fileName;
    }
    else if (fileName.Contains("favourite"))
    {
        listOfContent[folderName].IsFavourite = true;
    }
    else if (fileName.Contains("description"))
    {
        listOfContent[folderName].Description = fileName;
    }
    else if(fileName.Contains(".glb"))
    {
        listOfContent[folderName].Source = u;
    }
    else if (fileName.Contains("Environment"))
    {
        listOfContent[folderName].EnvironmentImages.Add(u);
    }
    else if (fileName.Contains("Thumbnail"))
    {
        listOfContent[folderName].Thumbnail = u;
    }
    else if (fileName.Contains("Skybox"))
    {
        listOfContent[folderName].SkyboxImages.Add(u);
    }
    else if (fileName.Contains("Animation"))
    {
        var animationsName = await new HttpClient().GetStringAsync(u);
        var animationName = animationsName.Split('\n').Select(x=>x.Trim()).Where(x=>x.Length>0).ToList();
        listOfContent[folderName].AnimationNames= animationName;
    }
}
listOfContent.Values.ToList()

In [ ]:
using System.Text.Json.Serialization;
var text = JsonSerializer.Serialize(listOfContent.Values.ToList(), new JsonSerializerOptions
{
    WriteIndented = true
});
text

In [ ]:
File.WriteAllText("index.json", text);